In [1]:
import pandas as pd
import numpy as np

import httpimport 

from pathlib import Path 

In [2]:
# Import personal library
with httpimport.github_repo('junclemente', 'jcds', ref='master'):
    import jcds.Quality as jq

In [3]:
datasets = Path('../datasets/')
df_file = 'maven_business_school/MavenBusinessSchool.xlsx' 
df = pd.read_excel(datasets/df_file)
df.head()

,Student ID,Undergrad Degree,Undergrad Grade,MBA Grade,Work Experience,Employability (Before),Employability (After),Status,Annual Salary
0,1,Business,68.4,90.2,No,252.0,276.0,Placed,111000
1,2,Business,62.1,92.8,No,423.0,410.0,Not Placed,NaN
2,3,Computer Science,70.2,68.7,Yes,101.0,119.0,Placed,107000
3,4,Engineering,75.1,80.7,No,288.0,334.0,Not Placed,NaN
4,5,Finance,60.9,74.9,No,248.0,252.0,Not Placed,NaN


In [4]:
df.shape

(1200, 9)

In [5]:
jq.dqr_cat(df)

The categorical features are: 
['Undergrad Degree', 'Work Experience', 'Status', 'Annual Salary']
Data Quality Report for Categorical Features
Stats
-----
            Feature  Count  Missing  % Missing  Cardinality
0  Undergrad Degree   1200        0       0.00            5
1   Work Experience   1200        0       0.00            2
2            Status   1200        0       0.00            2
3     Annual Salary    933      267      28.62           70


Mode 1
------
            Feature    Mode 1  Mode 1 Freq.  Mode 1 %
0  Undergrad Degree  Business           257     21.42
1   Work Experience        No          1066     88.83
2            Status    Placed           716     59.67
3     Annual Salary                     246     26.37


Mode 2
------
            Feature            Mode 2  Mode 2 Freq.  Mode 2 %
0  Undergrad Degree  Computer Science           240     20.00
1   Work Experience               Yes           134     11.17
2            Status        Not Placed           484     4

In [6]:
jq.dqr_cont(df)

The non-categorical features are: 
['Student ID', 'Undergrad Grade', 'MBA Grade', 'Employability (Before)', 'Employability (After)']
Data Quality for Continous Features
Total Features: 5
                  Feature  Count  Missing  % missing  Cardinality
0              Student ID   1200        0       0.00         1200
1         Undergrad Grade   1164       36       3.09          762
2               MBA Grade   1200        0       0.00          837
3  Employability (Before)   1193        7       0.59         1182
4   Employability (After)   1195        5       0.42          776


Descriptive Stats
                         count    mean     std   min     25%     50%     75%  \
Student ID              1200.0  600.50  346.55   1.0  300.75  600.50  900.25   
Undergrad Grade         1164.0   56.56   22.31  10.0   39.88   56.91   75.02   
MBA Grade               1200.0   52.83   23.49   0.0   35.72   53.20   72.47   
Employability (Before)  1193.0  216.31   36.28  62.0  189.96  215.24  239.66 

In [8]:
dff_file = 'maven_business_school/MavenBusinessSchool(final).xlsx' 
dff = pd.read_excel(datasets/dff_file)
dff.head()

,Student ID,Undergrad Degree,Undergrad Grade,MBA Grade,Work Experience,Employability (Before),Employability (After),Status,Annual Salary
0,1,Business,68.4,90.2,No,252.0,276.0,Placed,111000.0
1,2,Business,62.1,92.8,No,423.0,410.0,Not Placed,NaN
2,3,Computer Science,70.2,68.7,Yes,101.0,119.0,Placed,107000.0
3,4,Engineering,75.1,80.7,No,288.0,334.0,Not Placed,NaN
4,5,Finance,60.9,74.9,No,248.0,252.0,Not Placed,NaN


In [12]:
dff.shape

(1200, 9)

In [9]:
jq.dqr_cat(dff)

The categorical features are: 
['Undergrad Degree', 'Work Experience', 'Status']
Data Quality Report for Categorical Features
Stats
-----
            Feature  Count  Missing  % Missing  Cardinality
0  Undergrad Degree   1200        0        0.0            5
1   Work Experience   1200        0        0.0            2
2            Status   1200        0        0.0            2


Mode 1
------
            Feature    Mode 1  Mode 1 Freq.  Mode 1 %
0  Undergrad Degree  Business           257     21.42
1   Work Experience        No          1066     88.83
2            Status    Placed           716     59.67


Mode 2
------
            Feature            Mode 2  Mode 2 Freq.  Mode 2 %
0  Undergrad Degree  Computer Science           240     20.00
1   Work Experience               Yes           134     11.17
2            Status        Not Placed           484     40.33


Descriptive Stats
-----------------
                 count unique       top  freq
Undergrad Degree  1200      5  Business   

In [11]:
jq.dqr_cont(dff)

The non-categorical features are: 
['Student ID', 'Undergrad Grade', 'MBA Grade', 'Employability (Before)', 'Employability (After)', 'Annual Salary']
Data Quality for Continous Features
Total Features: 6
                  Feature  Count  Missing  % missing  Cardinality
0              Student ID   1200        0       0.00         1200
1         Undergrad Grade   1164       36       3.09          762
2               MBA Grade   1200        0       0.00          837
3  Employability (Before)   1193        7       0.59         1182
4   Employability (After)   1195        5       0.42          776
5           Annual Salary    716      484      67.60           89


Descriptive Stats
                         count       mean       std      min        25%  \
Student ID              1200.0     600.50    346.55      1.0     300.75   
Undergrad Grade         1164.0      56.56     22.31     10.0      39.88   
MBA Grade               1200.0      52.83     23.49      0.0      35.72   
Employability 